####  Deliverable 2. Create a Customer Travel Destinations Map

Use input statements to retrieve customer weather preferences, then use those preferences to identify potential travel destinations and nearby hotels. Then, show those destinations on a marker layer map with pop-up markers.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,72.28,77,20,8.05,few clouds
1,1,Albany,US,42.6001,-73.9662,69.03,84,98,1.99,overcast clouds
2,2,Provideniya,RU,64.3833,-173.3000,37.38,78,31,0.92,scattered clouds
3,3,Saldanha,ZA,-33.0117,17.9442,62.65,78,0,12.53,clear sky
4,4,Turukhansk,RU,65.8167,87.9833,31.73,65,83,3.89,broken clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
cities_df.head(10) 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,72.28,77,20,8.05,few clouds
9,9,Dharur,IN,18.8167,76.1167,71.47,86,100,10.13,overcast clouds
17,17,Vaini,TO,-21.2000,-175.2000,73.56,88,75,21.39,broken clouds
20,20,Vao,NC,-22.6667,167.4833,73.90,80,0,10.00,clear sky
24,24,Castro,BR,-24.7911,-50.0119,70.47,65,87,2.42,overcast clouds
26,26,Rikitea,PF,-23.1203,-134.9692,72.82,73,0,14.41,clear sky
51,51,Mahebourg,MU,-20.4081,57.7000,71.26,88,75,12.66,broken clouds
56,56,Hibbing,US,47.4272,-92.9377,71.56,77,0,0.00,clear sky
57,57,Puerto Ayora,EC,-0.7393,-90.3518,73.36,93,100,10.00,overcast clouds
59,59,Souillac,MU,-20.5167,57.5167,70.18,88,75,12.66,light rain


In [6]:
# 4a. Determine if there are any empty rows.
cities_df.count()

City_ID                63
City                   63
Country                63
Lat                    63
Lng                    63
Max Temp               63
Humidity               63
Cloudiness             63
Wind Speed             63
Current Description    63
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,72.28,few clouds,19.7297,-155.0900,
9,Dharur,IN,71.47,overcast clouds,18.8167,76.1167,
17,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,
20,Vao,NC,73.90,clear sky,-22.6667,167.4833,
24,Castro,BR,70.47,overcast clouds,-24.7911,-50.0119,
...,...,...,...,...,...,...,...
522,Komsomolskiy,UZ,74.12,few clouds,40.4272,71.7189,
528,Sarh,TD,73.31,moderate rain,9.1429,18.3923,
535,Katesh,TZ,71.67,clear sky,-4.5167,35.3833,
540,Aksay,RU,74.25,overcast clouds,47.2600,39.8700,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !='']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,72.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Dharur,IN,71.47,overcast clouds,18.8167,76.1167,Hotel Rajmahal
17,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
20,Vao,NC,73.90,clear sky,-22.6667,167.4833,Hotel Kou-Bugny
24,Castro,BR,70.47,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
26,Rikitea,PF,72.82,clear sky,-23.1203,-134.9692,People ThankYou
51,Mahebourg,MU,71.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
56,Hibbing,US,71.56,clear sky,47.4272,-92.9377,Mitchell-Tappan House - Carriage House Rentals
57,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
59,Souillac,MU,70.18,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
vacation_df = pd.read_csv('WeatherPy_Vacation.csv')

vacation_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Hilo,US,72.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,9,Dharur,IN,71.47,overcast clouds,18.8167,76.1167,Hotel Rajmahal
2,17,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,20,Vao,NC,73.90,clear sky,-22.6667,167.4833,Hotel Kou-Bugny
4,24,Castro,BR,70.47,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
5,26,Rikitea,PF,72.82,clear sky,-23.1203,-134.9692,People ThankYou
6,51,Mahebourg,MU,71.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
7,56,Hibbing,US,71.56,clear sky,47.4272,-92.9377,Mitchell-Tappan House - Carriage House Rentals
8,57,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,59,Souillac,MU,70.18,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [13]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>

"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store it in a new DataFrame
locations = vacation_df[['Lat', 'Lng']]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))